In [1]:
!pip install psycopg2-binary 
!pip install scikit-learn

In [2]:
from query_API import *

import pandas as pd
import dask.dataframe as dd

In [18]:
def percentage_diff(df):
    return (df['price_shift']-df['price']) / df['price'] * 100

In [57]:
def computer(**kwargs):
    
    # if start_roundid is specified, then compute from the start_roundid to latest_roundid in chainlink_real_time table
    if 'start_roundid' in kwargs:
        start_roundid = kwargs['start_roundid']
        start_roundid = str(start_roundid)
        query = "SELECT roundid, price, updatedat, pair, decimals FROM chainlink_real_time WHERE roundid > {}"
        formatted_query = query.format(start_roundid)
        df = query_from_db(formatted_query)
        
    # if nothing is specified, then compute all the data in chainlink_history and chainlink_real_time
    else: 
        history_df = query_from_db("Select roundid, price, updatedat, pair, decimals From chainlink_history")
        real_time_df = query_from_db("Select roundid, price, updatedat, pair, decimals From chainlink_real_time")
        df = pd.concat([history_df, real_time_df])
    
    # create a price_shift column that contains the price column that shift up one row
    df['price_shift'] = df['price'].shift(-1)
    
    # pandas dataframe to dask dataframe
    ddf = dd.from_pandas(df, npartitions=10)
    
    # calculate the percentage difference using the map_partitions() method
    ddf['percentage_diff'] = ddf.map_partitions(percentage_diff, meta=('x', 'f8'))

    # drop the first row, which will have NaN values
    ddf = ddf.dropna()
    
    # drop the price_shift column
    ddf = ddf.drop(columns=['price_shift'])

    # convert object to float and round to the second decimal points
    ddf['percentage_diff'] = ddf['percentage_diff'].astype(float).round(2)

    # save dask dataframe to pandas dataframe
    compute_df = ddf.compute()
        
    return compute_df

In [58]:
compute_history_df = computer()

In [59]:
compute_history_df.sort_values(by=['roundid'])

,roundid,price,updatedat,pair,decimals,percentage_diff
0,92233720368547771281,4652109401959,2021-12-31 22:13:16,BTC / USD,8,-0.27
1,92233720368547771282,4639357085285,2021-12-31 22:55:11,BTC / USD,8,-0.08
2,92233720368547771283,4635816164458,2021-12-31 23:56:36,BTC / USD,8,0.55
3,92233720368547771284,4661544706500,2022-01-01 00:42:51,BTC / USD,8,0.19
4,92233720368547771285,4670395911240,2022-01-01 00:56:01,BTC / USD,8,-0.01
...,...,...,...,...,...,...
1034,92233720368547792981,2924971428596,2023-04-28 16:55:23,BTC / USD,8,-0.04
1035,92233720368547792982,2923692000000,2023-04-28 17:55:23,BTC / USD,8,0.03
1036,92233720368547792983,2924547000000,2023-04-28 18:55:35,BTC / USD,8,0.59
1037,92233720368547792984,2941947320290,2023-04-28 19:04:23,BTC / USD,8,-0.19


In [60]:
compute_real_time_df = computer(start_roundid=92233720368547792981)

In [61]:
compute_real_time_df

,roundid,price,updatedat,pair,decimals,percentage_diff
0,92233720368547792982,2923692000000,2023-04-28 17:55:23,BTC / USD,8,0.03
1,92233720368547792983,2924547000000,2023-04-28 18:55:35,BTC / USD,8,0.59
2,92233720368547792984,2941947320290,2023-04-28 19:04:23,BTC / USD,8,-0.19
3,92233720368547792985,2936307000000,2023-04-28 19:55:35,BTC / USD,8,0.05


In [96]:
real_time = query_from_db("Select * From chainlink_real_time Order by roundid desc Limit 5")

In [97]:
real_time

,roundid,price,startedat,updatedat,answeredinround,pair,decimals
0,92233720368547793092,2809812000000,2023-05-01 23:55:35,2023-05-01 23:55:35,92233720368547793092,BTC / USD,8
1,92233720368547793091,2801721000000,2023-05-01 22:55:47,2023-05-01 22:55:47,92233720368547793091,BTC / USD,8
2,92233720368547793090,2799588300000,2023-05-01 21:55:35,2023-05-01 21:55:35,92233720368547793090,BTC / USD,8
3,92233720368547793089,2791877950000,2023-05-01 21:33:11,2023-05-01 21:33:11,92233720368547793089,BTC / USD,8
4,92233720368547793088,2776613450000,2023-05-01 20:55:35,2023-05-01 20:55:35,92233720368547793088,BTC / USD,8


In [98]:
compute = query_from_db("Select * From chainlink_compute Order by roundid desc Limit 5")

In [99]:
compute

,roundid,price,updatedat,pair,decimals,percentage_diff
0,92233720368547793089,2791877950000,2023-05-01 21:33:11,BTC / USD,8,0.28
1,92233720368547793088,2776613450000,2023-05-01 20:55:35,BTC / USD,8,0.55
2,92233720368547793087,2785014747523,2023-05-01 19:55:47,BTC / USD,8,-0.30
3,92233720368547793086,2800701000000,2023-05-01 19:28:11,BTC / USD,8,-0.56
4,92233720368547793085,2785959400000,2023-05-01 19:18:11,BTC / USD,8,0.53
